In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/29 20:07:16 INFO SparkContext: Running Spark version 2.4.3
21/11/29 20:07:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/29 20:07:17 INFO SparkContext: Submitted application: 6de845dd-81a7-4c91-9a9d-ee8cf2f24bea
21/11/29 20:07:17 INFO SecurityManager: Changing view acls to: oscar
21/11/29 20:07:17 INFO SecurityManager: Changing modify acls to: oscar
21/11/29 20:07:17 INFO SecurityManager: Changing view acls groups to: 
21/11/29 20:07:17 INFO SecurityManager: Changing modify acls groups to: 
21/11/29 20:07:17 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
21/11/29 20:07:17 INFO Utils: Successfully started service 'sparkDrive

21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:46861/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1638212837958
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:46861/jars/mainargs_2.12-0.1.4.jar with timestamp 1638212837958
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:46861/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1638212837958
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:46861/jars/os-lib_2.12-0.7.1.j

21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:46861/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1638212837963
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:46861/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1638212837963
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:46861/jars/argonaut_2.12-6.2.3.jar with timestamp 1638212837963
21/11/29 20:07:17 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                  

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@6d146ef3
import spark.implicits._


defined function run

In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [21]:
val drivers = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/drivers.csv")

load at cmd20.sc:4

1 / 1

drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]

In [3]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .select("raceId", "year")
    .where(col("year") === 2021)

load at cmd2.sc:4

1 / 1

races: Dataset[Row] = [raceId: string, year: string]

In [9]:
val driverWindow = Window.partitionBy("driverId")
val seasonWindow = Window.partitionBy("year")

driverWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2884d4c3
seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@72aa5cd2

In [29]:
val raceDriverLapWindow = Window.partitionBy("driverId", "raceId").orderBy("lap")

raceDriverLapWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@efb5892

In [34]:
val overtakes = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .withColumn("position", col("position").cast(IntegerType)) 
    .where(col("driverId") === 1)
    .where(col("raceId") === 1053)
    .join(races, "raceId")
    .withColumn("positionNextLap", lag(col("position"), 1).over(raceDriverLapWindow))
//     .withColumn("lapPositionDelta", col("position"))
    .select("driverId", "lap", "position", "positionNextLap")
    .sort(col("lap"))
    .show

load at cmd33.sc:4

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd33.sc:12

4 / 4

show at cmd33.sc:12

64 / 64

+--------+---+--------+---------------+
|driverId|lap|position|positionNextLap|
+--------+---+--------+---------------+
|       1|  1|       2|           null|
|       1| 10|       2|              2|
|       1| 11|       2|              2|
|       1| 12|       2|              2|
|       1| 13|       2|              2|
|       1| 14|       2|              2|
|       1| 15|       2|              2|
|       1| 16|       2|              2|
|       1| 17|       2|              2|
|       1| 18|       2|              2|
|       1| 19|       2|              2|
|       1|  2|       2|              2|
|       1| 20|       2|              2|
|       1| 21|       2|              2|
|       1| 22|       2|              2|
|       1| 23|       2|              2|
|       1| 24|       2|              2|
|       1| 25|       2|              2|
|       1| 26|       2|              2|
|       1| 27|       1|              2|
+--------+---+--------+---------------+
only showing top 20 rows



In [28]:
val results = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
//     .where(col("driverId") === 830)
    .withColumn("position", col("position").cast(IntegerType))    
    .withColumn("grid", col("grid").cast(IntegerType))    
    .withColumn("points", col("points").cast(IntegerType))

    .join(races, "raceId")

    .withColumn("averagePoints", round(avg(col("points")).over(driverWindow), 2))
    .withColumn("maxAvgPoints", max(col("averagePoints")).over(seasonWindow))
    .withColumn("pointPercent", round(col("averagePoints") / col("maxAvgPoints"), 2))
    .withColumn("podium", when(col("position") === 1 || col("position") === 2 ||col("position") === 3, lit(1)).otherwise(lit(0)))
    .withColumn("podiumPercent", round(sum(col("podium")).over(driverWindow) / count(col("podium")).over(driverWindow), 2))
    .withColumn("totalPodiums", sum(col("podium")).over(driverWindow))
    .withColumn("positionDelta", round(avg(col("position") - col("grid")).over(driverWindow), 2))
    .withColumn("champPoints", sum(col("points")).over(driverWindow))
    .join(drivers, "driverId")
    .dropDuplicates(Seq("driverId"))
    .select("code", "champPoints", "averagePoints", "pointPercent", "totalPodiums", "podiumPercent", "positionDelta")
    .sort(col("champPoints").desc)
    .show()

load at cmd27.sc:4

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

...

show at cmd27.sc:23

64 / 64

show at cmd27.sc:23

64 / 64

+----+-----------+-------------+------------+------------+-------------+-------------+
|code|champPoints|averagePoints|pointPercent|totalPodiums|podiumPercent|positionDelta|
+----+-----------+-------------+------------+------------+-------------+-------------+
| VER|        182|        20.22|         1.0|           8|         0.89|        -0.38|
| HAM|        150|        16.67|        0.82|           6|         0.67|         1.33|
| PER|        104|        11.56|        0.57|           2|         0.22|         0.33|
| NOR|        101|        11.22|        0.55|           3|         0.33|        -1.78|
| BOT|         92|        10.22|        0.51|           5|         0.56|          0.0|
| LEC|         62|         6.89|        0.34|           0|          0.0|          1.0|
| SAI|         60|         6.67|        0.33|           1|         0.11|        -0.33|
| RIC|         40|         4.44|        0.22|           0|          0.0|        -2.33|
| GAS|         39|         4.33|        0.2